## RECOMENDACIÓN DE ARTÍCULOS POR CESTA

In [1]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from joblib import dump, load

In [2]:
cestas = pd.read_csv('../data/processed/cestas.csv') 

In [3]:
productos = pd.read_csv('../data/processed/productos.csv') 

El modelo de recomendación usa una matriz, ya calculada con TfidfVectorizer

In [8]:
def recomienda_tfid(new_basket):
    # Cargar la matriz TF-IDF y el modelo
    tfidf_matrix = load('../models/tfidf_matrix.joblib')
    tfidf = load('../models/tfidf_model.joblib')

    # Convertir la nueva cesta en formato TF-IDF
    new_basket_str = ' '.join(new_basket)
    new_basket_tfidf = tfidf.transform([new_basket_str])

    # Comparar la nueva cesta con las anteriores
    similarities = cosine_similarity(new_basket_tfidf, tfidf_matrix)

    # Obtener los índices de las cestas más similares
    similar_indices = similarities.argsort()[0][-3:]  # Las 3 más similares

    # Crear un diccionario para contar las recomendaciones
    recommendations_count = {}
    total_similarity = 0

    # Recomendar productos de cestas similares
    for idx in similar_indices:
        sim_score = similarities[0][idx]
        total_similarity += sim_score
        products = cestas.iloc[idx]['Cestas'].split()
        
        for product in products:
            if product.strip() not in new_basket:  # Evitar recomendar lo que ya está en la cesta
                recommendations_count[product.strip()] = recommendations_count.get(product.strip(), 0) + sim_score

    # Calcular la probabilidad relativa de cada producto recomendado
    recommendations_with_prob = []
    if total_similarity > 0:  # Verificar que total_similarity no sea cero
        recommendations_with_prob = [(product, score / total_similarity) for product, score in recommendations_count.items()]
    else:
        print("No se encontraron similitudes suficientes para calcular probabilidades.")

    recommendations_with_prob.sort(key=lambda x: x[1], reverse=True)  # Ordenar por puntuación

    # Crear un nuevo DataFrame para almacenar las recomendaciones
    recommendations_data = []
    
    for product, prob in recommendations_with_prob:
        # Buscar la descripción en el DataFrame de productos
        description = productos.loc[productos['ARTICULO'] == product, 'DESCRIPCION']
        if not description.empty:
            recommendations_data.append({
                'ARTICULO': product,
                'DESCRIPCION': description.values[0],  # Obtener el primer valor encontrado
                'PROBABILIDAD': prob
            })

    recommendations_df = pd.DataFrame(recommendations_data)
    
    return recommendations_df


In [9]:
def recomienda_count(new_basket):
    # Cargar la matriz de conteo y el modelo desde la carpeta 'models'
    count_matrix = load('../models/count_matrix.joblib')
    count_vectorizer = load('../models/count_vectorizer.joblib')

    # Convertir la nueva cesta en formato de conteo
    new_basket_str = ' '.join(new_basket)
    new_basket_count = count_vectorizer.transform([new_basket_str])

    # Comparar la nueva cesta con las anteriores
    similarities = cosine_similarity(new_basket_count, count_matrix)

    # Obtener los índices de las cestas más similares
    similar_indices = similarities.argsort()[0][-3:]  # Las 3 más similares

    # Crear un diccionario para contar las recomendaciones
    recommendations_count = {}
    total_similarity = 0

    # Recomendar productos de cestas similares
    for idx in similar_indices:
        sim_score = similarities[0][idx]
        total_similarity += sim_score
        products = cestas.iloc[idx]['Cestas'].split()
        
        for product in products:
            if product.strip() not in new_basket:  # Evitar recomendar lo que ya está en la cesta
                recommendations_count[product.strip()] = recommendations_count.get(product.strip(), 0) + sim_score

    # Calcular la probabilidad relativa de cada producto recomendado
    recommendations_with_prob = []
    if total_similarity > 0:  # Verificar que total_similarity no sea cero
        recommendations_with_prob = [(product, score / total_similarity) for product, score in recommendations_count.items()]
    else:
        print("No se encontraron similitudes suficientes para calcular probabilidades.")

    recommendations_with_prob.sort(key=lambda x: x[1], reverse=True)  # Ordenar por puntuación

    # Crear un nuevo DataFrame para almacenar las recomendaciones
    recommendations_data = []
    
    for product, prob in recommendations_with_prob:
        # Buscar la descripción en el DataFrame de productos
        description = productos.loc[productos['ARTICULO'] == product, 'DESCRIPCION']
        if not description.empty:
            recommendations_data.append({
                'ARTICULO': product,
                'DESCRIPCION': description.values[0],  # Obtener el primer valor encontrado
                'PROBABILIDAD': prob
            })

    recommendations_df = pd.DataFrame(recommendations_data)
    
    return recommendations_df

# Prueba con TFID

In [13]:
# Ejemplo de una nueva cesta para recomendar productos.
# Cesta de prueba
new_basket = ['08049011087 ', '08049011087 ','0803B1103000', '0803B1103000']

In [14]:
recomendacion_tfid = recomienda_tfid(new_basket)
recomendacion_tfid

,ARTICULO,DESCRIPCION,PROBABILIDAD
0,08049011087,CODO DE PVC PARA ENCOLAR DE 110 87º MACHO HEMB...,19.466688
1,0660047,GRIFO FLOTADOR CON BOYA PARA CISTERNA DE WC AL...,3.430750
2,08041311087,TE DE PVC DE 110 87º MACHO HEMBRA GRIS,0.684327
3,013823401,MONOMANDO DE BAÑO DUCHA ENKEL CROMADO REF. 234...,0.343075
4,013823402,MONOMANDO DE DUCHA ENKEL CROMADO REF. 23402 CI...,0.343075
5,0201FC6328,F 63 MANGUITO ( ENLACE ) PARA COBRE COMPRESIÓN...,0.343075
6,0605DG,BOTE DE DECAPANTE CON PINCEL 100 GRAMOS PARA S...,0.343075
7,06076,ROLLO ESTAÑO PLATA TIPO 6%,0.343075
8,0719VCD3048CS1,PANEL FIJO DE DUCHA Y BARRA SOPORTE NIDIA 3 DE...,0.343075
9,04856053034,TERMOSTATO DIGITAL FILAR TYBOX 21 CON TECLAS P...,0.314762


# Prueba con COUNT

In [15]:
recomendacion_count = recomienda_count(new_basket)
recomendacion_count

,ARTICULO,DESCRIPCION,PROBABILIDAD
0,08049011087,CODO DE PVC PARA ENCOLAR DE 110 87º MACHO HEMB...,19.249295
1,0660047,GRIFO FLOTADOR CON BOYA PARA CISTERNA DE WC AL...,3.385763
2,08041311087,TE DE PVC DE 110 87º MACHO HEMBRA GRIS,0.672417
3,013823401,MONOMANDO DE BAÑO DUCHA ENKEL CROMADO REF. 234...,0.338576
4,013823402,MONOMANDO DE DUCHA ENKEL CROMADO REF. 23402 CI...,0.338576
5,0201FC6328,F 63 MANGUITO ( ENLACE ) PARA COBRE COMPRESIÓN...,0.338576
6,0605DG,BOTE DE DECAPANTE CON PINCEL 100 GRAMOS PARA S...,0.338576
7,06076,ROLLO ESTAÑO PLATA TIPO 6%,0.338576
8,0719VCD3048CS1,PANEL FIJO DE DUCHA Y BARRA SOPORTE NIDIA 3 DE...,0.338576
9,04856053034,TERMOSTATO DIGITAL FILAR TYBOX 21 CON TECLAS P...,0.325215
